In [1]:
import os, time
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.14.0


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.expand_dims(x_train, -1) #(60000, 28, 28, 1)
x_test = np.expand_dims(x_test, -1)

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  return model

# GPU

In [4]:
start=time.time()
model = create_model()
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
    metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=10
)

model.save_weights('./fashion_mnist.h5', overwrite=True)
print(f'{time.time()-start}s taken')

W0729 20:27:35.704040 139799312914304 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 17s 284us/sample - loss: 0.6776 - sparse_categorical_accuracy: 0.7826 - val_loss: 0.4149 - val_sparse_categorical_accuracy: 0.8496
Epoch 2/10
60000/60000 [==============================] - 15s 251us/sample - loss: 0.4047 - sparse_categorical_accuracy: 0.8562 - val_loss: 0.3127 - val_sparse_categorical_accuracy: 0.8837
Epoch 3/10
60000/60000 [==============================] - 15s 251us/sample - loss: 0.3676 - sparse_categorical_accuracy: 0.8705 - val_loss: 0.3068 - val_sparse_categorical_accuracy: 0.8901
Epoch 4/10
60000/60000 [==============================] - 15s 253us/sample - loss: 0.3410 - sparse_categorical_accuracy: 0.8807 - val_loss: 0.2896 - val_sparse_categorical_accuracy: 0.8955
Epoch 5/10
60000/60000 [==============================] - 15s 252us/sample - loss: 0.3166 - sparse_categorical_accuracy: 0.8863 - val_loss: 0.2941 - val_sparse_categorical_accuracy: 0.8970
Epoch

# TPU

In [4]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('Not using TPU')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print(f'{len(devices)} devices:')
  for d in devices: print(d)

TPU address is grpc://10.34.100.2:8470
11 devices:
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 9908414365363191594)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4974917026288295601)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7998588864879986827)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 15565643528276174813)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 17190374650284067929)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 7987109689500302738)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 2310587764381307383)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 11327104696702171434)
_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 14475938025564265619)
_DeviceAtt

In [6]:
start=time.time()
resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_accuracy'])

model.fit(
    x_train.astype(np.float32), y_train.astype(np.float32),
    epochs=10,
    steps_per_epoch=60,
    validation_data=(x_test.astype(np.float32), y_test.astype(np.float32)),
    validation_freq=10
)

model.save_weights('./fashion_mnist.h5', overwrite=True)
print(f'{time.time()-start}s taken')

W0729 20:40:27.111320 139820351559552 tpu_strategy_util.py:56] TPU system %s has already been initialized. Reinitializing the TPU can cause previously created variables on TPU to be lost.
W0729 20:40:46.720582 139820351559552 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training_distributed.py:411: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Epoch 1/10
60/60 [==============================] - 4s 70ms/step - loss: 1.0790 - sparse_categorical_accuracy: 0.6934
Epoch 2/10
60/60 [==============================] - 1s 23ms/step - loss: 0.5158 - sparse_categorical_accuracy: 0.8233
Epoch 3/10
60/60 [==============================] - 1s 23ms/step - loss: 0.4191 - sparse_categorical_accuracy: 0.8547
Epoch 4/10
60/60 [==============================] - 1s 23ms/step - loss: 0.3637 - sparse_categorical_accuracy: 0.8719
Epoch 5/10
60/60 [==============================] - 1s 23ms/step - loss: 0.3329 - sparse_categorical_accuracy: 0.8812
Epoch 6/10
60/60 [==============================] - 1s 23ms/step - loss: 0.3035 - sparse_categorical_accuracy: 0.8924
Epoch 7/10
60/60 [==============================] - 1s 23ms/step - loss: 0.2857 - sparse_categorical_accuracy: 0.8965
Epoch 8/10
60/60 [==============================] - 1s 23ms/step - loss: 0.2577 - sparse_categorical_accuracy: 0.9061
Epoch 9/10
60/60 [==============================] - 1s 2